In [167]:
from math import exp,log,sqrt
from scipy.stats import norm
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from riskmgmt import *
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm

Problem 1

In [8]:
S = 165
K = 165
r = 0.0425
q=0.0053
sigma = 0.2
T = (datetime(2022,4,15)-datetime(2022,3,13)).days/365

In [41]:
# Generalized Black Scholes Merton formula
def gbsm(S,K,T,r,b,sigma,call=True):
    d1 = (log(S/K)+(b+sigma**2/2)*T)/(sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    if call:
        return S * exp((b-r)*T) * norm(0,1).cdf(d1) - K*exp(-r*T)* norm(0,1).cdf(d2)
    else:
        return K*exp(-r*T)* norm(0,1).cdf(-d2) - S * exp((b-r)*T) * norm(0,1).cdf(-d1)

In [49]:
# Closed form Greeks for GBSM
def gbsm_greeks(S,K,T,r,b,sigma,call=True):
    d1 = (log(S/K)+(b+sigma**2/2)*T)/(sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    if call:
        delta = exp((b-r)*T) * norm(0,1).cdf(d1)
        theta = - S*exp((b-r)*T)*norm(0,1).pdf(d1)*sigma/(2*sqrt(T))- (b-r)*S*exp((b-r)*T)*norm(0,1).cdf(d1)-r*K*exp(-r*T)*norm(0,1).cdf(d2)
        rho = T*K*exp(-r*T)*norm(0,1).cdf(d2)
        carry_rho = T*S*exp((b-r)*T)*norm(0,1).cdf(d1)
    else:
        delta = exp((b-r)*T) * (norm(0,1).cdf(d1)-1)
        theta = - S*exp((b-r)*T)*norm(0,1).pdf(d1)*sigma/(2*sqrt(T))+ (b-r)*S*exp((b-r)*T)*norm(0,1).cdf(-d1)+ r*K*exp(-r*T)*norm(0,1).cdf(-d2)
        rho = -T*K*exp(-r*T)*norm(0,1).cdf(-d2)
        carry_rho = -T*S*exp((b-r)*T)*norm(0,1).cdf(-d1)
    gamma = norm(0,1).pdf(d1)* exp((b-r)*T)/ (S*sigma*sqrt(T))
    vega = S * exp((b-r)*T) * norm(0,1).pdf(d1) * sqrt(T)
    return delta,gamma,vega,theta,rho,carry_rho

In [50]:
# print out the greeks using closed form formula for call and put:
delta,gamma,vega,theta,rho,carry_rho = gbsm_greeks(current_price,K,T,r,r-q,sigma,True)
delta2,gamma2,vega2,theta2,rho2,carry_rho2 = gbsm_greeks(current_price,K,T,r,r-q,sigma,False)
print("Greeks for call:")
print("Delta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("carry_rho: {}".format(carry_rho))
print("\nGreeks for Put:")
print("Delta:{}".format(delta2))
print("gamma:{}".format(gamma2))
print("Vega: {}".format(vega2))
print("theta: {}".format(theta2))
print("rho: {}".format(rho2))
print("carry_rho: {}".format(carry_rho2))

Greeks for call:
Delta:0.5340091224850149
gamma:0.040037930803986446
Vega: 19.710179716477544
theta: -24.898522316969515
rho: 7.583586080244792
carry_rho: 7.966245676523029

Greeks for Put:
Delta:-0.4655118142202754
gamma:0.040037930803986446
Vega: 19.710179716477544
theta: -18.786996965277233
rho: -7.277010958127815
carry_rho: -6.944415968299725


In [70]:
# Finite difference dericative calculation
def fd_greeks(d_s,d_v,d_t,d_r,d_b,current_price,K,T,r,b,sigma,call):
    f_diff = (gbsm(current_price+d_s,K,T,r,b,sigma,call=call)-gbsm(current_price,K,T,r,b,sigma,call=call))/d_s
    b_diff = (gbsm(current_price,K,T,r,b,sigma,call=call)-gbsm(current_price-d_s,K,T,r,b,sigma,call=call))/d_s
    delta = f_diff
    gamma = (f_diff-b_diff)/d_s
    vega = (gbsm(current_price,K,T,r,b,sigma+d_v,call=call)-gbsm(current_price,K,T,r,b,sigma,call=call))/d_v
    theta = -(gbsm(current_price,K,T+d_t,r,b,sigma,call=call)-gbsm(current_price,K,T,r,b,sigma,call=call))/d_t
    rho = gbsm_greeks(current_price,K,T,r,b,sigma,call=call)[4]
    carry_rho = (gbsm(current_price,K,T,r,b+d_b,sigma,call=call)-gbsm(current_price,K,T,r,b,sigma,call=call))/d_b
    return delta,gamma,vega,theta,rho,carry_rho

In [71]:
# Print out finite difference for call and put
delta,gamma,vega,theta,rho,carry_rho=fd_greeks(0.2,0.01,0.01,0.01,0.007,current_price,K,T,r,r-q,sigma,True)
delta2,gamma2,vega2,theta2,rho2,carry_rho2=fd_greeks(0.2,0.01,0.01,0.01,0.007,current_price,K,T,r,r-q,sigma,False)
print("Finite Difference for call:")
print("Delta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("carry_rho: {}".format(carry_rho))
print("\nFinite Difference for Put:")
print("Delta:{}".format(delta2))
print("gamma:{}".format(gamma2))
print("Vega: {}".format(vega2))
print("theta: {}".format(theta2))
print("rho: {}".format(rho2))
print("carry_rho: {}".format(carry_rho2))

Finite Difference for call:
Delta:0.5380088534424488
gamma:0.04003661620579635
Vega: 19.71119469377811
theta: -24.322438934413526
rho: 7.583586080244792
carry_rho: 7.999955781329504

Finite Difference for Put:
Delta:-0.4615120832627184
gamma:0.04003661620615162
Vega: 19.71119469377811
theta: -18.212374651071173
rho: -7.277010958127815
carry_rho: -6.915425164275948


In [24]:
# binomial tree without dividend
def bt_american(S,K,T,r,b,sigma,N,call):
    dt = T/N
    u = exp(sigma*sqrt(dt))
    d = 1/u
    pu = (exp(b*dt)-d)/(u-d)
    pd = 1.0-pu
    df = exp(-r*dt)
    if call:
        z=1
    else:
        z=-1
    # calculate the number of nodes
    def nNode(n):
        return int((n+1)*(n+2)/2)
    # Calculate the index
    def idx(i,j):
        return nNode(j-1)+i
    nNodes = nNode(N)
    optionvalues = np.zeros(nNodes)
    for j in range(N,-1,-1):
        for i in range(j,-1,-1):
            index = idx(i,j)
            price = S*u**i*d**(j-i)
            optionvalues[index]=max(0,z*(price-K))
            if j<N:
               optionvalues[index] = max(optionvalues[index],df*(pu*optionvalues[idx(i+1,j+1)]+pd*optionvalues[idx(i,j+1)]))
            # print(i,j,optionvalues[index])
    return optionvalues[0]

In [26]:
# binomial tree with dividend
def bt_american_div(S,K,T,r,divamts,divtimes,sigma,N,call):
    if len(divamts)==0 or len(divtimes)==0:
        return bt_american(S,K,T,r,r,sigma,N,call)
    if divtimes[0]>N:
        return bt_american(S,K,T,r,r,sigma,N,call)
    dt = T/N
    u = exp(sigma*sqrt(dt))
    d = 1/u
    pu = (exp(r*dt)-d)/(u-d)
    pd = 1.0-pu
    df = exp(-r*dt)
    if call:
        z=1
    else:
        z=-1
    # calculate the number of nodes
    def nNode(n):
        return int((n+1)*(n+2)/2)
    # Calculate the index
    def idx(i,j):
        return nNode(j-1)+i
    nDiv = len(divtimes)
    nNodes = nNode(divtimes[0])
    optionvalues = np.zeros(nNodes)
    for j in range(divtimes[0],-1,-1):
        for i in range(j,-1,-1):
            index = idx(i,j)
            price = S*u**i*d**(j-i)
            if j < divtimes[0]:
                # Times before dividend, backward method
                optionvalues[index]=max(0,z*(price-K))
                optionvalues[index] = max(optionvalues[index],df*(pu*optionvalues[idx(i+1,j+1)]+pd*optionvalues[idx(i,j+1)]))
            else:
                valnoex = bt_american_div(price-divamts[0],K,T-divtimes[0]*dt,r,divamts[1:nDiv-1],divtimes[1:nDiv-1]-divtimes[0],sigma,N-divtimes[0],call)
                valex = max(0,z*(price-K))
                optionvalues[index] = max(valnoex,valex)
    return optionvalues[0]

In [27]:
S = 165
K = 165
N = 200
r = 0.0425
q=0.0053
sigma = 0.2
T = (datetime(2022,4,15)-datetime(2022,3,13)).days/365
divtimes = int((datetime(2022,4,11)-datetime(2022,3,13)).days/(datetime(2022,4,15)-datetime(2022,3,13)).days *N)
div=0.88

In [28]:
# Values without dividend
call_nodiv = bt_american(S,K,T,r,r,sigma,N,True)
put_nodiv = bt_american(S,K,T,r,r,sigma,N,False)
print("Call option without dividend:",call_nodiv)
print("Put option without dividend:",put_nodiv)
# Values with dividend
call_div = bt_american_div(S,K,T,r,np.array([div]),np.array([divtimes]),sigma,N,True)
put_div = bt_american_div(S,K,T,r,np.array([div]),np.array([divtimes]),sigma,N,False)


Call option without dividend: 4.2698585632362684
Put option without dividend: 3.684138176821656
Call option with dividend: 4.112836095267345
Put option with dividend: 4.1105345298444895


In [80]:
# Calculate greeks for bt
def fd_greeks_bt(d_s,d_v,d_t,d_r,d_d,S,K,T,r,divamts,divtimes,sigma,N,call):
    f_diff = (bt_american_div(S+d_s,K,T,r,divamts,divtimes,sigma,N,call)-bt_american_div(S,K,T,r,divamts,divtimes,sigma,N,call))/d_s
    b_diff = (bt_american_div(S,K,T,r,divamts,divtimes,sigma,N,call)-bt_american_div(S-d_s,K,T,r,divamts,divtimes,sigma,N,call))/d_s
    delta = f_diff
    gamma = (f_diff-b_diff)/d_s
    vega = (bt_american_div(S,K,T,r,divamts,divtimes,sigma+d_v,N,call)- bt_american_div(S,K,T,r,divamts,divtimes,sigma,N,call))/d_v
    theta = -(bt_american_div(S,K,T+d_t,r,divamts,divtimes,sigma,N,call)-bt_american_div(S,K,T,r,divamts,divtimes,sigma,N,call))/d_t
    rho = (bt_american_div(S,K,T,r+d_r,divamts,divtimes,sigma,N,call)-bt_american_div(S,K,T,r,divamts,divtimes,sigma,N,call))/d_r
    # The derivative respect to dividend
    ddiv = (bt_american_div(S,K,T,r,divamts,divtimes,sigma,N,call)-bt_american_div(S,K,T,r,divamts-d_d,divtimes,sigma,N,call))/d_r
    return delta,gamma,vega,theta,rho,ddiv

In [81]:
# Greeks without dividend
delta,gamma,vega,theta,rho,ddiv = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,S,K,T,r,np.array([]),np.array([]),sigma,N,True)
delta2,gamma2,vega2,theta2,rho2,ddiv2 = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,S,K,T,r,np.array([]),np.array([]),sigma,N,False)
print("Greeks for call:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("\nGreeks for Put:")
print("Deta:{}".format(delta2))
print("gamma:{}".format(gamma2))
print("Vega: {}".format(vega2))
print("theta: {}".format(theta2))
print("rho: {}".format(rho2))

Greeks for call:
Deta:0.07142098722151269
gamma:-1.8041124150158794e-14
Vega: 6.715925765729786
theta: -8.008532885682785
rho: 1.1144661756958774

Greeks for Put:
Deta:-0.9581440573525501
gamma:0.00879965784146286
Vega: 4.092784252978632
theta: -2.762962752360565
rho: -2.5389971559906854


In [82]:
# Greeks with dividend
delta,gamma,vega,theta,rho,ddiv = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,S,K,T,r,np.array([div]),np.array([divtimes]),sigma,N,True)
delta2,gamma2,vega2,theta2,rho2,ddiv2 = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,S,K,T,r,np.array([div]),np.array([1]),sigma,2,False)
print("Greeks for call:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("delta dividend : {}".format(ddiv))
print("\nGreeks for Put:")
print("Deta:{}".format(delta2))
print("gamma:{}".format(gamma2))
print("Vega: {}".format(vega2))
print("theta: {}".format(theta2))
print("rho: {}".format(rho2))
print("delta dividend : {}".format(ddiv2))

Greeks for call:
Deta:0.07142098722151269
gamma:-1.8041124150158794e-14
Vega: 6.715925765729786
theta: -8.008532885682785
rho: 1.1144661756958774
delta dividend : 0.0

Greeks for Put:
Deta:-0.9999999999999165
gamma:3.1086244689504383e-13
Vega: 0.0
theta: 3.5178104709400415
rho: -7.482600641783854
delta dividend : 0.9980806115089891


Problem 2

In [83]:
port_options = pd.read_csv("Problem2.csv")
daily_prices = pd.read_csv("DailyPrices.csv")

In [89]:
port_options['ExpirationDate']=port_options['ExpirationDate'].astype("datetime64[ns]")
port_options['OptionType'] = port_options['OptionType'].apply(lambda x: bool(x=="Call") if pd.notna(x) else x)

In [90]:
currdate = datetime(2023,3,3)
div_date = np.array([datetime(2023,3,15)])
r = 0.0425
T = (expiration-curr_date).days/365
N=50
divamts = np.array([1.00])
S =151.03

In [99]:
# Solve sigma using bt
def sigma_bt(S,K,T,r,divamts,divtimes,N,call,value,initvol):
    def sol_vol(x,S,K,T,r,divamts,divtimes,N,call,value):
        return bt_american_div(S,K,T,r,divamts,divtimes,x,N,call)-value
    vol = fsolve(sol_vol,initvol,args=(S,K,T,r,divamts,divtimes,N,call,value))
    return vol[0]

In [102]:
# Calculate implied vol
def row_fun_port(row,S,currdate,div_date,divamts,r,N):
    T = ((row['ExpirationDate']-currdate).days)/365
    divtime_cal = lambda x: int((x- currdate).days/(row['ExpirationDate']-currdate).days *N)
    divtimes = np.array([divtime_cal(d) for d in div_date])
    vol = sigma_bt(S,row['Strike'],T,r,divamts,divtimes,N,row['OptionType'],row['CurrentPrice'],0.2)
    return vol

In [103]:
port_options['Implied Vol'] = port_options[port_options['Type']=='Option'].apply(row_fun_port,args=(S,currdate,div_date,divamts,r,N),axis=1)

In [115]:
# Calculate the value for each portfolio
def portvalue(row,S,currdate,div_date,divamts,r,N,daysahead):
    # # Create a dataframe to store the values
    # port_value = pd.DataFrame(portdf['Portfolio']).copy()
    # port_value['value'] = np.NaN
    # # for each asset calculate the value using bt
    # Calculate the value using the currentdate moving ahead
    currdate += timedelta(days=daysahead)
    if row['Type']=='Stock':
            return S*row['Holding']
    else:
            T = ((row['ExpirationDate']-currdate).days)/365
            divtime_cal = lambda x: int((x- currdate).days/(row['ExpirationDate']-currdate).days *N)
            divtimes = np.array([divtime_cal(d) for d in div_date])
            value = bt_american_div(S,row['Strike'],T,r,divamts,divtimes,row['Implied Vol'],N,row['OptionType'])
            return value * row['Holding']

In [105]:
# Calculate the log return of AAPL
aapl_log = np.log(daily_prices['AAPL']/daily_prices['AAPL'].shift(1))
# Demean the series
aapl_h = aapl_log-aapl_log.mean()

In [106]:
# Simulate the price using normal
def normal_sim(r,ndays,p0,nsim=1000,seed=10):
    sigma = r.std()
    np.random.seed(seed)
    rsim = np.random.normal(0,sigma,(nsim,ndays))
    rsim_cum = np.sum(rsim,axis=1)
    psim = np.zeros(nsim)
    for i in range(nsim):
        psim[i]=p0*exp(rsim_cum[i])
    return psim

In [110]:
# Calculate delta for each asset
def asset_delta(row,S,currdate,div_date,divamts,r,N,d):
    if row['Type']=='Stock':
        return 1
    else:
        T = ((row['ExpirationDate']-currdate).days)/365
        divtime_cal = lambda x: int((x- currdate).days/(row['ExpirationDate']-currdate).days *N)
        divtimes = np.array([divtime_cal(d) for d in div_date])
        delta = (bt_american_div(S+d,row['Strike'],T,r,divamts,divtimes,row['Implied Vol'],N,row['OptionType'])- bt_american_div(S,row['Strike'],T,r,divamts,divtimes,row['Implied Vol'],N,row['OptionType']))/d
        return delta

In [113]:
port_options['delta'] = port_options.apply(asset_delta,args=(S,currdate,div_date,divamts,r,N,0.2),axis=1)
port_options['delta_h'] = port_options['delta']*port_options['Holding']
port_options['pv'] = port_options['CurrentPrice']*port_options['Holding']
price_sim = normal_sim(aapl_h,10,151.03,nsim=1000,seed=10)

In [116]:
# Calculate the PL for each simulation
# The current value of portfolios
port_curr = port_options.apply(portvalue,args=(151.03,currdate,div_date,divamts,r,N,0),axis=1)
pl_list = []
for i in range(len(price_sim)):
    pl = port_options.apply(portvalue,args=(price_sim[i],currdate,div_date,divamts,r,N,10),axis=1) - port_curr
    pl_list.append(pl)
pl_sim = pd.concat(pl_list,axis=1)
pl_sim.set_index(port_options['Portfolio'],inplace=True)
portpl_sim = pl_sim.groupby(level=0).sum().T

In [127]:
# Calculate Mean, VaR and ES
port_mean = portpl_sim.mean(axis=0)
stat_mean = pd.DataFrame(port_mean,columns=['Mean'])
def delta_var_es(row):
    r_sim = np.random.normal(0,row['scaler'],10000)
    var = VaR(r_sim)*row['pv']
    es = ES(r_sim)*row['pv']
    return var,es
sigma = aapl_h.var()
port_r_cal = port_options[['Portfolio','pv','delta_h']].groupby('Portfolio').sum()
port_r_cal['dr'] = S/port_r_cal['pv']*port_r_cal['delta_h']
port_r_cal['scaler']= np.sqrt(port_r_cal['dr']*sigma*port_r_cal['dr'])
port_r_cal['VaR'] = port_r_cal.apply(lambda x : delta_var_es(x)[0],axis=1)
port_r_cal['ES'] = port_r_cal.apply(lambda x : delta_var_es(x)[1],axis=1)
stat = pd.concat([stat_mean,port_r_cal[['VaR','ES']]],axis=1)
print(stat)

                  Mean        VaR         ES
Portfolio                                   
Call          0.364191   2.412045   2.973048
CallSpread   -0.468312   3.165946   4.021637
CoveredCall  -1.496461  11.147061  13.957524
ProtectedPut  1.334180   3.957125   4.877468
Put           0.948173   2.376212   2.923318
PutSpread     0.373035   1.246215   1.550986
Stock         0.548978   5.585891   6.885389
Straddle      1.312364   4.757654   6.126322
SynLong      -0.583983   0.003567   0.004534


Problem 3

In [184]:
research = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
momentum = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})

factor = (research.join(momentum, how='right') / 100).loc['2013-1-31':]
prices = pd.read_csv('DailyPrices.csv', parse_dates=['Date'])
all_returns = pd.DataFrame(return_calculate(prices)).set_index('Date')
stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
dataset = all_returns[stocks].join(factor)

subset = dataset.dropna()

In [196]:
X = subset[factors]
X = sm.add_constant(X)
y = subset[stocks].sub(subset['RF'], axis=0)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])

for stock in stocks:
    model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']

sub_return = pd.DataFrame(np.dot(factor[factors],betas.T), index=factor.index, columns=betas.index)
merge_return = pd.merge(sub_return,factor['RF'], left_index=True, right_index=True)
daily_expected_returns = merge_return.add(merge_return['RF'],axis=0).drop('RF',axis=1).add(alphas.T.loc['Alpha'], axis=1)
expected_annual_return = ((daily_expected_returns+1).cumprod().tail(1) ** (1/daily_expected_returns.shape[0]) - 1) * 252
expected_annual_return

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2023-01-31,0.157144,0.017941,0.2538,0.222901,0.155944,0.279721,0.120591,0.076962,-0.042945,0.129923,0.08154,0.521821,-0.033253,0.098273,0.241054,-0.155372,-0.017075,0.124206,-0.112301,0.147807


In [197]:
covariance_matrix = dataset[stocks].cov() * 252
covariance_matrix

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.126877,0.139557,0.037447,0.081272,0.102937,0.171265,0.066193,0.032745,0.122117,0.055520,0.036945,0.037700,0.154880,0.058646,0.071406,0.087399,0.111906,0.022740,0.066245,0.066123
META,0.139557,0.400843,0.017102,0.102465,0.142255,0.240599,0.098845,0.045091,0.194794,0.061619,0.033637,0.020759,0.173121,0.073973,0.085022,0.127047,0.182074,0.021329,0.088791,0.076719
UNH,0.037447,0.017102,0.060922,0.031117,0.036318,0.046531,0.026045,0.032068,0.034737,0.028173,0.027861,0.026843,0.039128,0.033321,0.029590,0.022467,0.029806,0.022981,0.034744,0.028847
MA,0.081272,0.102465,0.031117,0.095762,0.079856,0.137369,0.056792,0.033440,0.096194,0.047520,0.031163,0.030837,0.097521,0.058309,0.082400,0.076987,0.079016,0.017382,0.063399,0.051734
MSFT,0.102937,0.142255,0.036318,0.079856,0.127839,0.175956,0.070916,0.035082,0.133856,0.052676,0.033859,0.031304,0.131911,0.056714,0.068230,0.088227,0.120259,0.020323,0.065226,0.060804
NVDA,0.171265,0.240599,0.046531,0.137369,0.175956,0.403814,0.112055,0.046362,0.221364,0.084290,0.041944,0.054455,0.291392,0.098399,0.118145,0.157837,0.188012,0.021758,0.113397,0.098663
HD,0.066193,0.098845,0.026045,0.056792,0.070916,0.112055,0.097074,0.033271,0.096833,0.042310,0.034414,0.015936,0.077754,0.043555,0.050164,0.063936,0.069626,0.022364,0.046487,0.048412
PFE,0.032745,0.045091,0.032068,0.033440,0.035082,0.046362,0.033271,0.070517,0.037274,0.031202,0.027470,0.019979,0.022283,0.031862,0.030650,0.024522,0.029355,0.027603,0.030812,0.029531
AMZN,0.122117,0.194794,0.034737,0.096194,0.133856,0.221364,0.096833,0.037274,0.242775,0.065772,0.030153,0.037446,0.187717,0.070819,0.083373,0.124253,0.149924,0.022616,0.085241,0.071419
BRK-B,0.055520,0.061619,0.028173,0.047520,0.052676,0.084290,0.042310,0.031202,0.065772,0.050175,0.024839,0.034026,0.061541,0.047076,0.042080,0.051383,0.056281,0.019263,0.050820,0.040421


In [198]:
def super_efficient_portfolio(returns, rf_rate, cov_matrix):
    num_assets = returns.shape[0] if len(returns.shape) == 1 else returns.shape[1]
    
    def neg_sharpe_ratio(weights):
        port_return = np.sum(returns * weights)
        port_std_dev = np.sqrt(weights @ cov_matrix @ weights.T)
        sharpe_ratio = (port_return - rf_rate) / port_std_dev
        return -sharpe_ratio
    
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]
    
    bounds = [(0, 1) for _ in range(num_assets)]
    
    # Solve for optimal weights
    init_weights = np.ones(num_assets) / num_assets
    opt_result = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    opt_weights = opt_result.x * 100
    opt_port_return = np.sum(returns * opt_weights)
    opt_port_std_dev = np.sqrt(opt_weights @ cov_matrix @ opt_weights.T)
    opt_sharpe_ratio = (opt_port_return - rf_rate) / opt_port_std_dev
    return opt_weights, opt_sharpe_ratio

In [199]:
weights, sharpe_ratio = super_efficient_portfolio(expected_annual_return.values[0], 0.0425, covariance_matrix)
print("The Portfolio's Sharpe Ratio is: {:.2f}" .format(sharpe_ratio))
weights = pd.DataFrame(weights, index=expected_annual_return.columns, columns=['weight %']).round(2).T
weights

The Portfolio's Sharpe Ratio is: 1.65


,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
weight %,0.0,0.0,22.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.44,0.0,0.0,12.93,0.0,0.0,7.05,0.0,0.0
